In [1]:
import pandas as pd
import os
import threading
from tqdm import tqdm

In [2]:
def filtrar_linhas(arquivo_csv, filtro_EMI, filtro_DES, caminho_novo_csv):
    resultado = arquivo_csv[(arquivo_csv['MUNICIPIO_EMI'] == filtro_EMI) | (arquivo_csv['MUNICIPIO_DES'] == filtro_DES)]
    resultado.to_csv(caminho_novo_csv, sep=";", index=False, mode='w', header=True)

In [3]:
diretorio_caminho = r"D:\Notas Fiscais 2018"
diretorio = os.listdir(diretorio_caminho)

filtro_EMI, filtro_DES = "FORTALEZA", "FORTALEZA"
workers = 20

In [4]:
for item in diretorio:
    if item.endswith(".csv"):
        caminho_csv = os.path.join(diretorio_caminho, item)
        csv = pd.read_csv(caminho_csv, sep = ";")

        diretorio_novo_caminho = os.path.join(os.path.join(diretorio_caminho, r'Resultados'), item.split('.')[0])
        os.makedirs(diretorio_novo_caminho, exist_ok=True)

        tamanho_arg = len(csv) // workers
        threads = []

        for worker in tqdm(range(workers)):
            limite_inferior = worker * tamanho_arg
            limite_superior = (worker + 1) * tamanho_arg if worker != workers - 1 else len(csv)

            arg = csv[limite_inferior:limite_superior]
            caminho_thread_csv = os.path.join(diretorio_novo_caminho, f"worker_{worker + 1}.csv")

            t = threading.Thread(target=filtrar_linhas, args=(arg, filtro_EMI, filtro_DES, caminho_thread_csv))
            threads.append(t)
            t.start()

        for thread in threads:
            thread.join()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 32.80it/s]


In [7]:
caminho_resultados = r"D:\Notas Fiscais 2018\Resultados"
resultados = os.listdir(caminho_resultados)

In [8]:
for item in resultados:
    if not item.endswith(".csv"):
        caminho_csvs = os.path.join(caminho_resultados, item)
        csvs = os.listdir(caminho_csvs)

        csv_final_caminho = os.path.join(caminho_resultados, item+"_filtrado.csv")

        lista_csvs = []
        
        for csv in csvs:
            caminho_parte = os.path.join(caminho_csvs, csv)
            parte = pd.read_csv(caminho_parte, sep = ";")
            lista_csvs.append(parte)

        merged_csv = pd.concat(lista_csvs, ignore_index=True)
        merged_csv.to_csv(csv_final_caminho, sep=";", index=False, mode='w', header=True)